12 steps to Navier-Stokes
=====
***

This lesson complements the first interactive module of the online [CFD Python](https://bitbucket.org/cfdpython/cfd-python-class) class, by Prof. Lorena A. Barba, called **12 Steps to Navier-Stokes.** It was written with BU graduate student Gilbert Forsyth.

Array Operations with NumPy
----------------

For more computationally intensive programs, the use of built-in Numpy functions can provide an  increase in execution speed many-times over.  As a simple example, consider the following equation:

$$u^{n+1}_i = u^n_i-u^n_{i-1}$$

Now, given a vector $u^n = [0, 1, 2, 3, 4, 5]\ \ $   we can calculate the values of $u^{n+1}$ by iterating over the values of $u^n$ with a for loop.  

In [10]:
u = [0, 1, 2, 3, 4, 5]

for i in 2:length(u)
    print(u[i]-u[i-1],"\n")
end

1
1
1
1
1


This is the expected result and the execution time was nearly instantaneous.  If we perform the same operation as an array operation, then rather than calculate $u^n_i-u^n_{i-1}\ $ 5 separate times, we can slice the $u$ array and calculate each operation with one command:

In [15]:
u[2:end]-u[1:end-1]

5-element Array{Int64,1}:
 1
 1
 1
 1
 1

What this command says is subtract the 1st, 2nd, 3rd, 4th and 5th elements of $u$ from the 2nd, 3rd, 4th, 5th and 6th elements of $u$.  

### Speed Increases

For a 6 element array, the benefits of array operations are pretty slim.  There will be no appreciable difference in execution time because there are so few operations taking place.  But if we revisit 2D linear convection, we can see some substantial speed increases.  


In [18]:
nx = 81;
ny = 81;
nt = 100;
c = 1;
dx = 2/(nx-1);
dy = 2/(ny-1);
sigma = .2;
dt = sigma*dx;

x = linspace(0,2,nx);
y = linspace(0,2,ny);

u = ones((ny,nx)) ##create a 1xn vector of 1's
un = ones((ny,nx));##

###Assign initial conditions

u[Int(.5/dy):Int(1/dy+1),Int(.5/dx):Int(1/dx+1)]=2;

time it

In [31]:
tic();
u = ones((ny,nx))
u[Int(.5/dy):Int(1/dy+1),Int(.5/dx):Int(1/dx+1)]=2;

for n in 0:nt ##loop across number of time steps
    un = copy(u);
    row, col = size(u);
    for j in 2: row-1
        for i in 2:col-1
            u[j,i] = un[j, i] - (c*dt/dx*(un[j,i] - un[j,i-1]))-(c*dt/dy*(un[j,i]-un[j-1,i]))
            u[1,:] = 1
            u[end,:] = 1
            u[:,1] = 1
            u[:,end] = 1
        end
    end
end
toc()

elapsed time: 0.706641236 seconds


0.706641236

With the "raw" Python code above, the best execution time achieved was 1.83 seconds.  Keep in mind that with these three nested loops, that the statements inside the **j** loop are being evaluated more than 650,000 times.   Let's compare that with the performance of the same code implemented with array operations:

In [34]:
tic();
u = ones((ny,nx))
u[Int(.5/dy):Int(1/dy+1),Int(.5/dx):Int(1/dx+1)]=2;

for n in 0:nt ##loop across number of time steps
    un = copy(u)
    u[2:end,2:end]=un[2:end,2:end]-(c*dt/dx*(un[2:end,1:end-1]-un[2:end, 1:end-1]))-(c*dt/dy*(un[2:end,2:end]-un[1:end-1,2:end]))
    u[1,:] = 1
    u[end,:] = 1
    u[:,1] = 1
    u[:,] = 1
end
toc()

elapsed time: 0.198117472 seconds


0.198117472

As you can see, the speed increase is substantial.  The same calculation goes from 0.71 seconds to 0.20 seconds.  It isn't a huge amount of time to wait, but these speed gains will increase exponentially with the size and complexity of the problem being evaluated.  

In [7]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()